# Lab/HomeWork  01: Comparison of background segmentation algorithms


## Introduction
In this notebook a brief study on the performance of different background segmentation algorithms in different environments and situations will be carried out. Specifically, we will use videos from the `CDNet` [1] dataset of 2012 and 2014 that for reasons of time limitation to the framework of a weekly laboratory practice have been selected by hand seeking to have a selection of videos that represent different situations and environments.

The following is the code that will compute the algorithms `MOG` [2], `GMG` [3], `CNT` [4], `LSBP` [5] and `GSOC` [6],from the `OpenCV` library for background segmentation (Only the algorithms within the ``bgsegm``). The results, an analysis and conclusions will be included afterwards.


### Code

In [1]:
import cv2 as cv
import glob
import numpy as np
import os
import time

ALGORITHMS_TO_EVALUATE = [
    (cv.bgsegm.createBackgroundSubtractorMOG, 'MOG', {}),
    (cv.bgsegm.createBackgroundSubtractorGMG, 'GMG', {}),
    (cv.bgsegm.createBackgroundSubtractorCNT, 'CNT', {}),
    (cv.bgsegm.createBackgroundSubtractorLSBP, 'LSBP-vanilla', {'nSamples': 20, 'LSBPRadius': 4, 'Tlower': 2.0, 'Tupper': 200.0, 'Tinc': 1.0, 'Tdec': 0.05, 'Rscale': 5.0, 'Rincdec': 0.05, 'LSBPthreshold': 8}),
    (cv.bgsegm.createBackgroundSubtractorLSBP, 'LSBP-speed', {'nSamples': 10, 'LSBPRadius': 16, 'Tlower': 2.0, 'Tupper': 32.0, 'Tinc': 1.0, 'Tdec': 0.05, 'Rscale': 10.0, 'Rincdec': 0.005, 'LSBPthreshold': 8}),
    (cv.bgsegm.createBackgroundSubtractorLSBP, 'LSBP-quality', {'nSamples': 20, 'LSBPRadius': 16, 'Tlower': 2.0, 'Tupper': 32.0, 'Tinc': 1.0, 'Tdec': 0.05, 'Rscale': 10.0, 'Rincdec': 0.005, 'LSBPthreshold': 8}),
    (cv.bgsegm.createBackgroundSubtractorLSBP, 'LSBP-camera-motion-compensation', {'mc': 1}),
    (cv.bgsegm.createBackgroundSubtractorGSOC, 'GSOC', {}),
    (cv.bgsegm.createBackgroundSubtractorGSOC, 'GSOC-camera-motion-compensation', {'mc': 1})
]

def contains_relevant_files(root):
    return os.path.isdir(os.path.join(root, 'groundtruth')) and os.path.isdir(os.path.join(root, 'input'))


def find_relevant_dirs(root):
    relevant_dirs = []
    for d in sorted(os.listdir(root)):
        d = os.path.join(root, d)
        if os.path.isdir(d):
            if contains_relevant_files(d):
                relevant_dirs += [d]
            else:
                relevant_dirs += find_relevant_dirs(d)
    return relevant_dirs


def load_sequence(root):
    gt_dir, frames_dir = os.path.join(root, 'groundtruth'), os.path.join(root, 'input')
    gt = sorted(glob.glob(os.path.join(gt_dir, '*.png')))
    f = sorted(glob.glob(os.path.join(frames_dir, '*.jpg')))
    assert(len(gt) == len(f))
    return gt, f


def evaluate_algorithm(gt, frames, algo, algo_arguments, output_folder=None):
    bgs = algo(**algo_arguments)
    mask = []
    t_start = time.time()

    for i in range(len(gt)):
        frame = np.uint8(cv.imread(frames[i], cv.IMREAD_COLOR))
        mask.append(bgs.apply(frame))

    average_duration = (time.time() - t_start) / len(gt)
    average_precision, average_recall, average_f1, average_accuracy = [], [], [], []

    # Save output
    if output_folder is not None:
        if not os.path.isdir(output_folder):
            os.makedirs(output_folder)
        for i in range(len(mask)):
            cv.imwrite(os.path.join(output_folder, '%05d.png' % i), mask[i])

    for i in range(len(gt)):
        gt_mask = np.uint8(cv.imread(gt[i], cv.IMREAD_GRAYSCALE))
        roi = ((gt_mask == 255) | (gt_mask == 0))
        if roi.sum() > 0:
            gt_answer, answer = gt_mask[roi], mask[i][roi]

            tp = ((answer == 255) & (gt_answer == 255)).sum()
            tn = ((answer == 0) & (gt_answer == 0)).sum()
            fp = ((answer == 255) & (gt_answer == 0)).sum()
            fn = ((answer == 0) & (gt_answer == 255)).sum()

            if tp + fp > 0:
                average_precision.append(float(tp) / (tp + fp))
            if tp + fn > 0:
                average_recall.append(float(tp) / (tp + fn))
            if tp + fn + fp > 0:
                average_f1.append(2.0 * tp / (2.0 * tp + fn + fp))
            average_accuracy.append(float(tp + tn) / (tp + tn + fp + fn))

    return average_duration, np.mean(average_precision), np.mean(average_recall), np.mean(average_f1), np.mean(average_accuracy)


def evaluate_on_sequence(seq, summary, algos):
    gt, frames = load_sequence(seq)
    category, video_name = os.path.basename(os.path.dirname(seq)), os.path.basename(seq)
    print('=== %s:%s ===' % (category, video_name))
    
    for algo, algo_name, algo_arguments in algos:
        print('Algorithm: %s' % algo_name)
        sec_per_step, precision, recall, f1, accuracy = evaluate_algorithm(gt, frames, algo, algo_arguments, output_folder=os.path.join(seq, "output", algo_name))
        print('Average accuracy: %.3f' % accuracy)
        print('Average precision: %.3f' % precision)
        print('Average recall: %.3f' % recall)
        print('Average F1: %.3f' % f1)
        print('Average sec. per step: %.4f' % sec_per_step)
        print('')

        if category not in summary:
            summary[category] = {}
        if algo_name not in summary[category]:
            summary[category][algo_name] = []
        summary[category][algo_name].append((precision, recall, f1, accuracy))

    return summary

def main_evaluator(dataset_path, algorithms_to_evaluate):
    print(f'Input file path: {dataset_path}')
    
    # DS dirs
    dataset_dirs = find_relevant_dirs(dataset_path)
    assert len(dataset_dirs) > 0, ("Passed directory must contain at least one sequence from the Change Detection dataset. There is no relevant directories in %s. Check that this directory is correct." % (dataset_path))
    print(f"Relevant directories: {dataset_dirs}")

    # Algorithm
    global ALGORITHMS_TO_EVALUATE
    algos = []
    for algo, algo_name, algo_arguments in ALGORITHMS_TO_EVALUATE:
        if algo_name in algorithms_to_evaluate:
            algos.append((algo, algo_name, algo_arguments))

    summary = {}
    for seq in dataset_dirs:
        summary = evaluate_on_sequence(seq, summary, algos)

    for category in summary:
        for algo_name in summary[category]:
            summary[category][algo_name] = np.mean(summary[category][algo_name], axis=0)

    for category in summary:
        print('=== SUMMARY for %s (Precision, Recall, F1, Accuracy) ===' % category)
        for algo_name in summary[category]:
            print('%05s: %.3f %.3f %.3f %.3f' % ((algo_name,) + tuple(summary[category][algo_name])))

    return summary    



In [2]:
algos = [ x[1] for x in ALGORITHMS_TO_EVALUATE]
summary = main_evaluator('my_dataset/',algos)

Input file path: my_dataset/
Relevant directories: ['my_dataset/PTZ/zoomInZoomOut', 'my_dataset/badWeather/snowFall', 'my_dataset/baseline/highway', 'my_dataset/cameraJitter/traffic', 'my_dataset/dynamicBackground/fall', 'my_dataset/intermittentObjectMotion/abandonedBox', 'my_dataset/lowFramerate/port_0_17fps', 'my_dataset/nightVideos/tramStation', 'my_dataset/shadow/copyMachine', 'my_dataset/thermal/corridor', 'my_dataset/turbulence/turbulence2']
=== PTZ:zoomInZoomOut ===
Algorithm: MOG
Average accuracy: 0.962
Average precision: 0.235
Average recall: 0.528
Average F1: 0.198
Average sec. per step: 0.0027

Algorithm: GMG
Average accuracy: 0.867
Average precision: 0.234
Average recall: 0.997
Average F1: 0.300
Average sec. per step: 0.0066

Algorithm: CNT
Average accuracy: 0.363
Average precision: 0.003
Average recall: 0.774
Average F1: 0.005
Average sec. per step: 0.0012

Algorithm: LSBP-vanilla
Average accuracy: 0.719
Average precision: 0.007
Average recall: 0.904
Average F1: 0.013
Aver

/home/pedro/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pedro/.local/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Average accuracy: 0.992
Average precision: nan
Average recall: 0.000
Average F1: 0.000
Average sec. per step: 0.0282

Algorithm: LSBP-speed
Average accuracy: 0.983
Average precision: 0.249
Average recall: 0.684
Average F1: 0.241
Average sec. per step: 0.0248

Algorithm: LSBP-quality
Average accuracy: 0.992
Average precision: 0.425
Average recall: 0.346
Average F1: 0.215
Average sec. per step: 0.0290

Algorithm: LSBP-camera-motion-compensation
Average accuracy: 0.967
Average precision: 0.051
Average recall: 0.462
Average F1: 0.065
Average sec. per step: 0.0702

Algorithm: GSOC
Average accuracy: 0.997
Average precision: 0.611
Average recall: 0.661
Average F1: 0.510
Average sec. per step: 0.0165

Algorithm: GSOC-camera-motion-compensation
Average accuracy: 0.991
Average precision: 0.345
Average recall: 0.515
Average F1: 0.301
Average sec. per step: 0.0697

=== baseline:highway ===
Algorithm: MOG
Average accuracy: 0.964
Average precision: 0.869
Average recall: 0.354
Average F1: 0.492
Avera

In [4]:
from tabulate import tabulate

# Obtain formatted results (markdown)
for folder,metrics in summary.items():
    print(f"- {folder}\n")
    # print a in a md table format
    a = [ [k]+list(v) for k,v in metrics.items() ]
    print(tabulate(a, headers=['algo','precision','recall','f1','accuracy'], tablefmt='github'))

- PTZ

| algo                            |   precision |   recall |         f1 |   accuracy |
|---------------------------------|-------------|----------|------------|------------|
| MOG                             |  0.235231   | 0.527892 | 0.197583   |   0.961836 |
| GMG                             |  0.233798   | 0.996825 | 0.299962   |   0.86734  |
| CNT                             |  0.00255921 | 0.774085 | 0.00510058 |   0.363204 |
| LSBP-vanilla                    |  0.00679596 | 0.904434 | 0.0134843  |   0.719286 |
| LSBP-speed                      |  0.00548402 | 0.995884 | 0.0109042  |   0.614114 |
| LSBP-quality                    |  0.00675031 | 0.813981 | 0.0133803  |   0.736604 |
| LSBP-camera-motion-compensation |  0.00733403 | 0.847087 | 0.0145304  |   0.743451 |
| GSOC                            |  0.0120096  | 0.991169 | 0.0235153  |   0.74323  |
| GSOC-camera-motion-compensation |  0.0114699  | 0.990305 | 0.0225019  |   0.734258 |
- badWeather

| algo                

## 2. Methods

In the following, the different [Background Subtraction](https://docs.opencv.org/3.4/d1/dc5/tutorial_background_subtraction.html) methods to be used will be briefly described:

- [Mixture of Gausian (MOG)](https://docs.opencv.org/3.4/d2/d55/group__bgsegm.html#ga17d9525d2ad71f74d8d29c2c5e11903d): MOG models the background of a video sequence using a mixture of Gaussian distributions, which are updated over time to adapt to changes. It subtracts the current frame from the background model and threshold the result to detect moving objects. The MOG algorithm is effective but may produce false positives or fail to detect small or slow-moving objects.

- [Gausian Mixture model with adaptative learning rates - Godbehere, Matsukawa and Goldberg (GMG)](https://docs.opencv.org/4.x/d1/d5c/classcv_1_1bgsegm_1_1BackgroundSubtractorGMG.html) : This algorithm is an improved version of the MOG algorithm for background subtraction. It models the background of a video sequence as a mixture of Gaussian distributions and adapts the learning rate of each Gaussian based on the level of activity in the scene. 

- [Count (CNT)](https://docs.opencv.org/4.x/de/dca/classcv_1_1bgsegm_1_1BackgroundSubtractorCNT.html): Algorithm developed with simplicity in mind, trying to capture the essence of the background subtraction process as it is performed in human vision. The implementation of a simple algorithm is fast as it is, but was further optimized using practical software development methods.

- [Local SVD Binary Pattern (LSBP)](https://docs.opencv.org/4.x/de/d4c/classcv_1_1bgsegm_1_1BackgroundSubtractorLSBP.html): Due to illumination changes in different scenes and the effect of noise pixels, the background detection methods often result in false positives in complex environments. To solve this problem, LSBP proposes an adaptive background subtraction model that uses a new binary SVD pattern instead of simply relying on color intensity. This feature can describe the potential structure of local regions in a given image, therefore, it can improve robustness to illumination variations, noise and shadows. Specifically, different parameter settings will be used for this algorithm whose names are sufficiently self-descriptive (vanilla, speed, quality, camera-motion-compensation).

- [Google Summer of Code (GSOC)](https://docs.opencv.org/3.4/d4/dd5/classcv_1_1bgsegm_1_1BackgroundSubtractorGSOC.html): GSOC is an algorithm based on LSBP, which performs better in regions with varying illumination, noise and shadows. In other words, it is a faster and more noise resistant algorithm. One of the main changes is that after descriptor extraction LSBP processes the frames pixel by pixel (i.e. independently). This is the common feature among all consensus-based background models. This means that the implementation can take advantage of parallelism. Note the use of a Camera Motion Compensation oriented version of parameters.

## 3. Results

In this section the results of the different techniques employed in the different scenarios will be discussed. It should be noted that the `CDNet` dataset specifies the ROI, however it has not been employed, using the total image area for the error calculation (instead of areas of interest).

Therefore, a table with the results of the different algorithms for the different `CDNet` scenarios tested will be shown below for each environment. Additionally, the videos of the unsegmented environment (input), the ground truth and the result of the best algorithm (in bold) will be shown so that their performance can be visually estimated. It is important to note that the metric used to evaluate the results is the `F1-score` instead of the `accuracy`, as the latter is not a good metric for this type of problem. This is due to the high number of true negatives, i.e., the correct background classification makes it possible to obtain high values in this metric without being representative of a good or bad segmentation. On the other hand, the `F1-score` is the harmonic mean between precision and recall, which takes into account the number of false positives and negatives.

- **PTZ**: The best algorithm is `GMG`, but there is a clear problem with zoomOut, as it detects it as motion. Note that this problem is even more pronounced with other methods.

| algo                            |   precision |   recall |         f1 |   accuracy |
|---------------------------------|-------------|----------|------------|------------|
| MOG                             |  0.235231   | 0.527892 | 0.197583   |   0.961836 |
| GMG                             |  0.233798   | 0.996825 | **0.299962**   |   0.86734  |
| CNT                             |  0.00255921 | 0.774085 | 0.00510058 |   0.363204 |
| LSBP-vanilla                    |  0.00679596 | 0.904434 | 0.0134843  |   0.719286 |
| LSBP-speed                      |  0.00548402 | 0.995884 | 0.0109042  |   0.614114 |
| LSBP-quality                    |  0.00675031 | 0.813981 | 0.0133803  |   0.736604 |
| LSBP-camera-motion-compensation |  0.00733403 | 0.847087 | 0.0145304  |   0.743451 |
| GSOC                            |  0.0120096  | 0.991169 | 0.0235153  |   0.74323  |
| GSOC-camera-motion-compensation |  0.0114699  | 0.990305 | 0.0225019  |   0.734258 |

<video width="320" height="240" controls src="./video/ptz/input.mp4"></video>
<video width="320" height="240" controls src="./video/ptz/gt.mp4"></video>
<video width="320" height="240" controls src="./video/ptz/gmg.mp4"></video>

- **badWeather**: Similar to the previous environment, there are major problems with motion detection in snow particles, which introduce a large number of false positives, and the segmentation of cars (true positives) also leaves much to be desired. In this case, the best algorithm is `GSOC`. As a curiosity, `LSBP-vanilla` has a very poor performance, which has not been able to detect any movement (probably due to falling snow).


| algo                            |   precision |   recall |        f1 |   accuracy |
|---------------------------------|-------------|----------|-----------|------------|
| MOG                             |   0.409     | 0.28781  | 0.184691  |   0.99594  |
| GMG                             |   0.110096  | 0.370786 | 0.101978  |   0.988549 |
| CNT                             |   0.244927  | 0.457302 | 0.17062   |   0.994411 |
| LSBP-vanilla                    |   0         | 0        | 0         |   0.992029 |
| LSBP-speed                      |   0.249006  | 0.683939 | 0.241421  |   0.983495 |
| LSBP-quality                    |   0.424577  | 0.345561 | 0.215345  |   0.992429 |
| LSBP-camera-motion-compensation |   0.0505324 | 0.461697 | 0.0652106 |   0.967376 |
| GSOC                            |   0.610978  | 0.661407 | **0.510371**  |   0.997084 |
| GSOC-camera-motion-compensation |   0.34458   | 0.514632 | 0.300728  |   0.991395 |

<video width="320" height="240" controls src="./video/badWeather/input.mp4"></video>
<video width="320" height="240" controls src="./video/badWeather/gt.mp4"></video>
<video width="320" height="240" controls src="./video/badWeather/gsoc.mp4"></video>


- **baseline**: In general the performance of a few algorithms is really good, with `GMG`, `GSOC` and `GSOC-camera-motion-compensation` standing out. In this case, the best algorithm is `GSOC-camera-motion-compensation`, which is able to detect the movement of the cars with a precision that is just superior to `GSOC`. The drawbacks are the inclusion of very few false positives on the road and the non-perfect segmentation of the cars.

| algo                            |   precision |   recall |       f1 |   accuracy |
|---------------------------------|-------------|----------|----------|------------|
| MOG                             |    0.868912 | 0.353839 | 0.492289 |   0.964207 |
| GMG                             |    0.914775 | 0.995248 | 0.951883 |   0.993895 |
| CNT                             |    0.728345 | 0.834702 | 0.756585 |   0.977393 |
| LSBP-vanilla                    |    0.12038  | 0.59233  | 0.185195 |   0.764323 |
| LSBP-speed                      |    0.252615 | 0.965846 | 0.384239 |   0.848653 |
| LSBP-quality                    |    0.278128 | 0.862342 | 0.401256 |   0.875205 |
| LSBP-camera-motion-compensation |    0.278401 | 0.860373 | 0.40142  |   0.875449 |
| GSOC                            |    0.997483 | 0.96439  | 0.980033 |   0.997962 |
| GSOC-camera-motion-compensation |    0.997386 | 0.965061 | **0.980414** |   0.997955 |

<video width="320" height="240" controls src="./video/baseline/input.mp4"></video>
<video width="320" height="240" controls src="./video/baseline/gt.mp4"></video>
<video width="320" height="240" controls src="./video/baseline/gsoc-cmc.mp4"></video>


- **cameraJitter**: `GSOC` will be the best algorithm, however vibrations cause the results to be not quite acceptable despite proper segmentation of the cars (too many false positives, reflected in the accuracy).

| algo                            |   precision |    recall |        f1 |   accuracy |
|---------------------------------|-------------|-----------|-----------|------------|
| MOG                             |   0.57765   | 0.742257  | 0.545076  |   0.972512 |
| GMG                             |   0.272066  | 0.978759  | 0.363338  |   0.863264 |
| CNT                             |   0.217812  | 0.890286  | 0.30176   |   0.83299  |
| LSBP-vanilla                    |   0.0491276 | 0.0419539 | 0.0338397 |   0.890082 |
| LSBP-speed                      |   0.453129  | 0.842171  | 0.488248  |   0.948485 |
| LSBP-quality                    |   0.515379  | 0.536767  | 0.418203  |   0.953715 |
| LSBP-camera-motion-compensation |   0.174116  | 0.685294  | 0.218179  |   0.752431 |
| GSOC                            |   0.58472   | 0.953882  | **0.619819**  |   0.965874 |
| GSOC-camera-motion-compensation |   0.329962  | 0.856668  | 0.384673  |   0.848864 |

<video width="320" height="240" controls src="./video/cameraJitter/input.mp4"></video>
<video width="320" height="240" controls src="./video/cameraJitter/gt.mp4"></video>
<video width="320" height="240" controls src="./video/cameraJitter/gsoc.mp4"></video>


- **dynamicBackground**: All algorithms will again introduce a high number of false positives arising from the motion of the background (a tree in this case). In the face of bad results, `MOG`, `GSOC` and `GSOC-camera-motion-compensation` have shown better performance than the other algorithms, the last one being the best by a slight difference.

| algo                            |   precision |   recall |        f1 |   accuracy |
|---------------------------------|-------------|----------|-----------|------------|
| MOG                             |  0.16117    | 0.64193  | 0.165554  |   0.980667 |
| GMG                             |  0.0643223  | 0.967305 | 0.092016  |   0.853648 |
| CNT                             |  0.0362688  | 0.732975 | 0.0542603 |   0.731069 |
| LSBP-vanilla                    |  0.00931162 | 0.413278 | 0.0134723 |   0.612216 |
| LSBP-speed                      |  0.0312488  | 0.988431 | 0.050265  |   0.595335 |
| LSBP-quality                    |  0.0362564  | 0.982987 | 0.0574047 |   0.674216 |
| LSBP-camera-motion-compensation |  0.0359427  | 0.980615 | 0.0570143 |   0.672425 |
| GSOC                            |  0.133319   | 0.924511 | 0.166045  |   0.955389 |
| GSOC-camera-motion-compensation |  0.133416   | 0.918624 | **0.166085**  |   0.9554   |

<video width="320" height="240" controls src="./video/dynamicBackground/input.mp4"></video>
<video width="320" height="240" controls src="./video/dynamicBackground/gt.mp4"></video>
<video width="320" height="240" controls src="./video/dynamicBackground/gsoc-cmc.mp4"></video>


- **intermittentObjectMotion**: The results obtained are generally poor, with the exception of `CNT`, which obtains a `F1-score` of almost twice the number of competitors. As for the quality of the results, although the segmentation is not entirely bad, the large amount of noise intensifies the poor results. The other algorithms have problems with the modeling of the box, because it is an object that remains static and moving at times. This box is the main object to be segmented, together with the viandes.

| algo                            |   precision |   recall |       f1 |   accuracy |
|---------------------------------|-------------|----------|----------|------------|
| MOG                             |    0.651193 | 0.17361  | 0.216514 |   0.961351 |
| GMG                             |    0.550901 | 0.266283 | 0.265416 |   0.958658 |
| CNT                             |    0.371165 | 0.91466  | **0.519832** |   0.918778 |
| LSBP-vanilla                    |    0.222042 | 0.298234 | 0.246852 |   0.939625 |
| LSBP-speed                      |    0.312236 | 0.533082 | 0.34464  |   0.912814 |
| LSBP-quality                    |    0.3514   | 0.444092 | 0.30286  |   0.926341 |
| LSBP-camera-motion-compensation |    0.312311 | 0.438521 | 0.289585 |   0.925418 |
| GSOC                            |    0.785011 | 0.343284 | 0.342146 |   0.960699 |
| GSOC-camera-motion-compensation |    0.768172 | 0.334468 | 0.331519 |   0.960028 |

<video width="320" height="240" controls src="./video/intermittentObjectMotion/input.mp4"></video>
<video width="320" height="240" controls src="./video/intermittentObjectMotion/gt.mp4"></video>
<video width="320" height="240" controls src="./video/intermittentObjectMotion/cnt.mp4"></video>


- **lowFramerate**: The results are really bad due to the continuous and slight rocking of the ships, and the ship in motion is very poorly segmented. As a better algorithm, `MOG` stands out, even if its results are bad.

| algo                            |   precision |   recall |         f1 |   accuracy |
|---------------------------------|-------------|----------|------------|------------|
| MOG                             | 0.0283849   | 0.554928 | **0.0371217**  |   0.995732 |
| GMG                             | 0.00453097  | 0.979605 | 0.00860428 |   0.909679 |
| CNT                             | 0.000817868 | 0.690986 | 0.00161289 |   0.769969 |
| LSBP-vanilla                    | 0.00140222  | 0.835772 | 0.00274951 |   0.844888 |
| LSBP-speed                      | 0.00896049  | 0.875752 | 0.0162497  |   0.972289 |
| LSBP-quality                    | 0.0202978   | 0.76415  | 0.0324731  |   0.990614 |
| LSBP-camera-motion-compensation | 0.0202133   | 0.770388 | 0.0324456  |   0.990519 |
| GSOC                            | 0.0232342   | 0.734985 | 0.0332148  |   0.964558 |
| GSOC-camera-motion-compensation | 0.022149    | 0.737946 | 0.0322175  |   0.963959 |

<video width="320" height="240" controls src="./video/lowFramerate/input.mp4"></video>
<video width="320" height="240" controls src="./video/lowFramerate/gt.mp4"></video>
<video width="320" height="240" controls src="./video/lowFramerate/mog.mp4"></video>



- **nightVideos**: Con diferencia `LSBP-speed` consigue los mejores resultados. 

| algo                            |   precision |   recall |       f1 |   accuracy |
|---------------------------------|-------------|----------|----------|------------|
| MOG                             |    0.684792 | 0.517716 | 0.55089  |   0.980709 |
| GMG                             |    0.377    | 0.802826 | 0.479219 |   0.965064 |
| CNT                             |    0.511386 | 0.716514 | 0.557563 |   0.979821 |
| LSBP-vanilla                    |    0.483524 | 0.254811 | 0.274109 |   0.973345 |
| LSBP-speed                      |    0.6181   | 0.90353  | **0.712885** |   0.983055 |
| LSBP-quality                    |    0.684189 | 0.655568 | 0.622423 |   0.983232 |
| LSBP-camera-motion-compensation |    0.685357 | 0.656491 | 0.62352  |   0.98324  |
| GSOC                            |    0.644393 | 0.741551 | 0.664462 |   0.979677 |
| GSOC-camera-motion-compensation |    0.645074 | 0.744367 | 0.666729 |   0.97973  |

<video width="320" height="240" controls src="./video/nightVideos/input.mp4"></video>
<video width="320" height="240" controls src="./video/nightVideos/gt.mp4"></video>
<video width="320" height="240" controls src="./video/nightVideos/lsbp-speed.mp4"></video>


- **shadow**: As expected, shadows cause a large number of false positives. `LSBP-quality` achieves the best results.

| algo                            |   precision |   recall |       f1 |   accuracy |
|---------------------------------|-------------|----------|----------|------------|
| MOG                             |    0.870772 | 0.346188 | 0.412274 |   0.95009  |
| GMG                             |    0.530122 | 0.330694 | 0.329393 |   0.931434 |
| CNT                             |    0.516114 | 0.629644 | 0.494071 |   0.929194 |
| LSBP-vanilla                    |    0.957764 | 0.116136 | 0.201223 |   0.937954 |
| LSBP-speed                      |    0.695163 | 0.658132 | 0.601373 |   0.951795 |
| LSBP-quality                    |    0.833208 | 0.56837  | **0.603416** |   0.956846 |
| LSBP-camera-motion-compensation |    0.774321 | 0.566933 | 0.569282 |   0.952981 |
| GSOC                            |    0.715491 | 0.557841 | 0.517791 |   0.947252 |
| GSOC-camera-motion-compensation |    0.706334 | 0.553322 | 0.508953 |   0.946374 |

<video width="320" height="240" controls src="./video/shadow/input.mp4"></video>
<video width="320" height="240" controls src="./video/shadow/gt.mp4"></video>
<video width="320" height="240" controls src="./video/shadow/LSBP-quality.mp4"></video>


- **thermal**: The segmentations are good, as they are able to segment with adequate fidelity the area of interest in multiple algorithms. However, reflections cause a high rate of false positives in the recall. `GSOC` has the best trade-off between accuracy and recall.

| algo                            |   precision |    recall |       f1 |   accuracy |
|---------------------------------|-------------|-----------|----------|------------|
| MOG                             |    0.83986  | 0.243774  | 0.291908 |   0.982864 |
| GMG                             |    0.804782 | 0.0998715 | 0.135167 |   0.972637 |
| CNT                             |    0.339534 | 0.934827  | 0.401108 |   0.968101 |
| LSBP-vanilla                    |    0.302021 | 0.215863  | 0.105725 |   0.965191 |
| LSBP-speed                      |    0.357355 | 0.580965  | 0.396127 |   0.984226 |
| LSBP-quality                    |    0.246242 | 0.212073  | 0.158135 |   0.958591 |
| LSBP-camera-motion-compensation |    0.217422 | 0.278346  | 0.1404   |   0.952065 |
| GSOC                            |    0.675316 | 0.51434   | **0.455121** |   0.988736 |
| GSOC-camera-motion-compensation |    0.585561 | 0.476604  | 0.403153 |   0.982786 |

<video width="320" height="240" controls src="./video/thermal/input.mp4"></video>
<video width="320" height="240" controls src="./video/thermal/gt.mp4"></video>
<video width="320" height="240" controls src="./video/thermal/gsoc.mp4"></video>


- **turbulence**: Except for the rest of the algorithms the segmentation of `GSOC` and `GSOC-camera-motion-compensation` are really good. In particular, `GSOC` outperforms `GSOC-camera-motion-compensation` by a very small margin.

| algo                            |   precision |   recall |        f1 |   accuracy |
|---------------------------------|-------------|----------|-----------|------------|
| MOG                             |   0.193085  | 0.839513 | 0.220158  |   0.999334 |
| GMG                             |   0.386646  | 0.703433 | 0.336272  |   0.999848 |
| CNT                             |   0.0083218 | 0.923123 | 0.0161081 |   0.950858 |
| LSBP-vanilla                    |   0.0121646 | 0.982866 | 0.0233425 |   0.970296 |
| LSBP-speed                      |   0.0208108 | 0.993007 | 0.0384346 |   0.978657 |
| LSBP-quality                    |   0.0404582 | 0.984396 | 0.0686685 |   0.987851 |
| LSBP-camera-motion-compensation |   0.0401586 | 0.984254 | 0.0682421 |   0.987468 |
| GSOC                            |   0.959853  | 0.957874 | **0.943183**  |   0.999978 |
| GSOC-camera-motion-compensation |   0.95657   | 0.958499 | 0.941041  |   0.999977 |

<video width="320" height="240" controls src="./video/turbulence/input.mp4"></video>
<video width="320" height="240" controls src="./video/turbulence/gt.mp4"></video>
<video width="320" height="240" controls src="./video/turbulence/gsoc.mp4"></video>



## Discussion


In general, the performance of the algorithms is affected by complex backgrounds as well as videos with stability deficiencies, which make the background modeling notoriously difficult. However, it should be noted the good performance, robustness and versatility of the `GSOC` algorithm and its parameter variant `GSOC-camera-motion-compensation` which has managed in many occasions to outperform the rest (and in those that have not, it has not lagged behind at all). As for algorithms such as `MOG`, `GMG` or `CNT`, although they have managed to be the best in one environment each, the results have been generally bad, not being able to say that in these environments in which they have been the best their behavior was at any time ideal (for example `GMG` in zoomOut is the best, but the results are bad). Finally, as for `LSBP`, specifically its variants `LSBP-speed` and `LSBP-quality`, it must be said that they have managed to outperform the rest in two cases with acceptable results (nightVideos and shadow, in this last one the shadow has been a problem even though it has been attenuated).
 
Regarding execution times, as mentioned in the introduction to the methods section, `CNT` is undoubtedly the fastest, followed by `MOG`. As for the slowest, it is undoubtedly the `LSBP` family. Thus, considering the quality of the results and the execution time, `GSOC` is considered one of the best options for real-time video segmentation using background modeling methods (`GSOC-camera-motion-compensation` is much slower and the results are very similar).



## References

- [1] Y. Wang, P. -M. Jodoin, F. Porikli, J. Konrad, Y. Benezeth and P. Ishwar, "CDnet 2014: An Expanded Change Detection Benchmark Dataset," 2014 IEEE Conference on Computer Vision and Pattern Recognition Workshops, Columbus, OH, USA, 2014, pp. 393-400, doi: 10.1109/CVPRW.2014.126.

- [2] Pakorn KaewTraKulPong and Richard Bowden. An improved adaptive background mixture model for real-time tracking with shadow detection. In Video-Based Surveillance Systems, pages 135–144. Springer, 2002.

- [3] Andrew B Godbehere, Akihiro Matsukawa, and Ken Goldberg. Visual tracking of human visitors under variable-lighting conditions for a responsive audio art installation. In American Control Conference (ACC), 2012, pages 4305–4312. IEEE, 2012.

- [4] Algorithm by Sagi Zeevi - https://github.com/sagi-z/BackgroundSubtractorCNT 

- [5] L. Guo, D. Xu, and Z. Qiang. Background subtraction using local svd binary pattern. In 2016 IEEE Conference on Computer Vision and Pattern Recognition Workshops (CVPRW), pages 1159–1167, June 2016.

- [6] GSOC - https://summerofcode.withgoogle.com/archive/2017/projects/6453014550282240 // https://github.com/opencv/opencv_contrib/pull/1211